In [3]:

from vcis.reverse_geocoder.reverse_geocoder_polygon import ReverseGeocoder
from vcis.databases.oracle.oracle_tools import OracleTools
oracle_tool = OracleTools()
gdf = oracle_tool.get_all_cities_geopandas()

In [35]:
import numpy as np

# gdf.to_file('sample_points.geojson', driver='GeoJSON')

print(np.sort(gdf["shapegroup"].unique()))
gdf.count()


['AFG' 'AGO' 'ALB' 'ARM' 'ASM' 'ATA' 'AZE' 'BDI' 'BEN' 'BFA' 'BGD' 'BGR'
 'BIH' 'BLR' 'BLZ' 'BOL' 'BRA' 'BTN' 'BWA' 'CAF' 'CHN' 'CIV' 'CMR' 'COD'
 'COG' 'COL' 'COM' 'CPV' 'CRI' 'CUB' 'DJI' 'DMA' 'DOM' 'DZA' 'ECU' 'EGY'
 'ERI' 'ETH' 'FJI' 'GAB' 'GEO' 'GGY' 'GHA' 'GIN' 'GMB' 'GNB' 'GNQ' 'GRD'
 'GTM' 'GUY' 'HND' 'HTI' 'IDN' 'IND' 'IRN' 'IRQ' 'JAM' 'JOR' 'KAZ' 'KEN'
 'KGZ' 'KHM' 'KIR' 'LAO' 'LBN' 'LBY' 'LCA' 'LKA' 'LSO' 'MAR' 'MDA' 'MDG'
 'MDV' 'MEX' 'MHL' 'MMR' 'MNE' 'MNG' 'MOZ' 'MRT' 'MUS' 'MWI' 'MYS' 'NAM'
 'NER' 'NGA' 'NIC' 'NPL' 'NRU' 'PAK' 'PCN' 'PER' 'PNG' 'PRY' 'PSE' 'ROU'
 'RUS' 'RWA' 'SLB' 'SLE' 'SLV' 'SOM' 'SRB' 'STP' 'SUR' 'SWZ' 'SYR' 'TCD'
 'TGO' 'THA' 'TJK' 'TKM' 'TLS' 'TON' 'TUN' 'TUR' 'TUV' 'TZA' 'UGA' 'UKR'
 'UZB' 'VCT' 'VEN' 'VNM' 'VUT' 'WSM' 'YEM' 'ZMB']


shapename     774371
shapeiso         324
shapeid       774432
shapegroup    774432
shapetype     774432
geometry      774432
dtype: int64

In [31]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os
import math
import folium
import warnings
from vcis.databases.cassandra import cassandra_tools
cached_results= {}


world = gdf
def reverse_geocode_point(point):
        if point in cached_results:
            return cached_results[point]
        
        possible_matches_idx = list(world.sindex.intersection(point.bounds))
        possible_matches = world.iloc[possible_matches_idx]
        country_info = possible_matches[possible_matches.geometry.contains(point)][['shapename', 'shapegroup']].to_dict(orient='records')
        
        if not country_info:
            closest_idx = world['geometry'].distance(point).idxmin()
            country_info = [{'shapename': world.loc[closest_idx, 'shapename'], 'shapegroup': world.loc[closest_idx, 'shapegroup']}]
        
        cached_results[point] = country_info
        return country_info

def reverse_geocode_multipoints(df, with_Binning=False):
    if with_Binning:
        points = [Point(xy) for xy in zip(df['longitude_grid'], df['latitude_grid'])]
    else:
        points = [Point(xy) for xy in zip(df['LOCATION_LONGITUDE'], df['LOCATION_LATITUDE'])]

    country_info = []
    for point in points:
        result = reverse_geocode_point(point)
        # Make sure the result dictionary has both 'NAME' and 'country_code' keys
        
        country_info.extend(result)
    return country_info


# def reverse_geocode_with_Binning( df, step_distance=200):
#     df = binning(df, step_distance)
#     country_names = self.reverse_geocode_multipoints(df, with_Binning=True)
#     return country_names
# def get_step_size(self, distance=1):
#     latitude = 33.8
#     one_degree_lat = 110.574
#     one_degree_lat_radians = math.radians(latitude)
#     one_degree_long = 111.320 * math.cos(one_degree_lat_radians)
#     step_lon = (distance / (one_degree_long * 1000))
#     step_lat = (distance / (one_degree_lat * 1000))
#     return step_lat, step_lon

# def binning_lat(self, data, step):
#     to_bin = lambda x: np.floor(x / step) * step
#     data.loc[:, "latitude_grid"] = data['LOCATION_LATITUDE'].apply(to_bin)
#     data.loc[:, "latitude_grid"] = data['latitude_grid'] + step / 2
#     return data

# def binning_lon(self, data, step):
#     to_bin = lambda x: np.floor(x / step) * step
#     data.loc[:, "longitude_grid"] = data['LOCATION_LONGITUDE'].apply(to_bin)
#     data.loc[:, "longitude_grid"] = data['longitude_grid'] + step / 2
#     return data

# def binning(self, data, distance=200):
#     step_lat, step_lon = self.get_step_size(distance=distance)
#     data = self.binning_lat(data, step_lat)
#     data = self.binning_lon(data, step_lon)
#     return data

In [33]:
point = Point(48.5, 35)
print("Reverse geocoding for a single point:")
print(reverse_geocode_point(point))


# Test reverse geocoding for multiple points
df = pd.DataFrame({'LOCATION_LONGITUDE': [-110, -100, -90,48,49,30,49,44,21,34], 'LOCATION_LATITUDE': [40, 30, 20,33,43,23,54,65,43,36]})
print("\nReverse geocoding for multiple points:")
print(reverse_geocode_multipoints(df))

Reverse geocoding for a single point:
[{'shapename': '?????? ???????', 'shapegroup': 'IRN'}]

Reverse geocoding for multiple points:
[{'shapename': 'Ascensión', 'shapegroup': 'MEX'}, {'shapename': 'Jiménez', 'shapegroup': 'MEX'}, {'shapename': 'Hecelchakán', 'shapegroup': 'MEX'}, {'shapename': '?????? ??????', 'shapegroup': 'IRN'}, {'shapename': 'Kayakentsky District', 'shapegroup': 'RUS'}, {'shapename': 'Paris Paris', 'shapegroup': 'EGY'}, {'shapename': 'Cherdaklinsky District', 'shapegroup': 'RUS'}, {'shapename': 'Pinezhsky District', 'shapegroup': 'RUS'}, {'shapename': 'Kursumlija Municipality', 'shapegroup': 'SRB'}, {'shapename': 'Silifke', 'shapegroup': 'TUR'}]
